Sabiendo los resultado anteriores, vamos a analizar los datos ahora imputandolos con datos reales

In [11]:
import pandas as pd
from datetime import datetime
from statistics import mode
import statsmodels.api as sm
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error

In [12]:
pd.set_option('display.max_rows', 500)
pd.set_option('display.max_columns', 500)
%matplotlib inline

In [13]:
df = pd.read_csv('data/TyT/Resultados_Saber_TyT_Gen_ricas_2020-1.csv')

C:\Users\David\AppData\Local\Temp\ipykernel_19616\1842402721.py:1: DtypeWarning: Columns (9,10,11,12,13,14,15,16,17,18,19) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv('data/TyT/Resultados_Saber_TyT_Gen_ricas_2020-1.csv')


In [14]:
def delete_columns(array=None, data_frame = pd.DataFrame):
    if array is None:
        array = []
    for column in array:
        data_frame = data_frame.drop(column, axis=1)
    return data_frame

In [15]:
drop_columns = [
    'ESTU_TIPODOCUMENTO',
    'ESTU_NACIONALIDAD',
    'ESTU_EXTERIOR',
    'PERIODO',
    'ESTU_CONSECUTIVO',
    'ESTU_ESTUDIANTE',
    'ESTU_PAIS_RESIDE',
    'ESTU_DISC_FISICA',
    'ESTU_DISC_SISTEM',
    'ESTU_DISC_AUDITCASTELL',
    'ESTU_DISC_AUDITLSC',
    'ESTU_DISC_VISUALCEGUE',
    'ESTU_DISC_VISUALBAJA',
    'ESTU_DISC_VOZHABLA',
    'ESTU_DISC_INTELEC',
    'ESTU_DISC_ESPAUTISTA',
    'ESTU_DISC_PSICOSOCIAL',
    'ESTU_DISC_ASPERGER',
    'ESTU_COD_RESIDE_DEPTO',
    'ESTU_COD_RESIDE_MCPIO',
    'ESTU_CODDANE_COLE_TERMINO',
    'ESTU_COD_COLE_MCPIO_TERMINO',
    'INST_COD_INSTITUCION',
    'ESTU_SNIES_PRGMACADEMICO',
    'ESTU_PRGM_CODMUNICIPIO',
    'ESTU_INST_CODMUNICIPIO',
    'ESTU_PRIVADO_LIBERTAD',
    'ESTU_COD_MCPIO_PRESENTACION',
    'ESTU_COD_DEPTO_PRESENTACION',
    'MOD_INGLES_DESEM',
    'ESTU_ESTADOINVESTIGACION',
    'ESTU_COLE_TERMINO',
    'ESTU_OTROCOLE_TERMINO'
]

In [16]:
df = delete_columns(drop_columns, df)

In [17]:
def convert_age(data_frame = pd.DataFrame):
    #Definimos la fecha por defecto que en este caso sera el dia en que se presentaron las pruebas ese aanio agosto 1
    fecha_default = datetime(2020,8,1)
    #Obtenemos solo la fecha
    data_frame = data_frame.replace(r'\s.*', '', regex=True)
    data_frame = pd.to_datetime(data_frame, format='%m/%d/%Y', errors='coerce')
    #Agregamos la fecha por defecto a los datos que no tengan fecha
    data_frame.fillna(fecha_default, inplace=True)
    #Calculamos la edad
    data_frame = (fecha_default - data_frame).dt.days // 365
    return  data_frame

In [18]:
df['ESTU_FECHANACIMIENTO'] = convert_age(df['ESTU_FECHANACIMIENTO'])
df.rename(columns={'ESTU_FECHANACIMIENTO': 'EDAD'}, inplace=True)

In [19]:
def imputar(dataframe = pd.DataFrame):
    for column in dataframe:
        if dataframe[column].dtype == 'O':
            find_mode = mode(dataframe[column])
            dataframe[column].fillna(find_mode, inplace=True)
        else:
            find_mean = dataframe[column].mean()
            dataframe[column].fillna(find_mean, inplace=True)
    return dataframe

In [20]:
df = imputar(df)

In [21]:
df['MCPIO_VIVE_ESTUDIA'] = df['ESTU_MCPIO_RESIDE'] == df['ESTU_PRGM_MUNICIPIO']

In [22]:
df_vive_estudia = pd.get_dummies(df[['MCPIO_VIVE_ESTUDIA']],prefix='MCPIO_VIVE_ESTUDIA').astype(int)
df['MCPIO_VIVE_ESTUDIA'] = df_vive_estudia

In [23]:
def group_categories(dataframe=pd.DataFrame):
    column_object = dataframe.select_dtypes(include=['object']).columns
    for item in column_object:
        if len(dataframe[item].unique()) > 4:
            quantiles = pd.qcut(dataframe[item].value_counts(), q=4, labels=['Grupo 1', 'Grupo 2', 'Grupo 3', 'Grupo 4'])
            quantiles = pd.DataFrame({item: quantiles.index, f'GROUP_{item}': quantiles.values})
            dataframe = dataframe.merge(quantiles, on=item, how='left')
            dataframe.drop(item, axis=1, inplace=True)
    return dataframe

In [24]:
df = group_categories(df)

In [25]:
def create_all_dummies(dataframe = pd.DataFrame):
    categoric_columns = dataframe.select_dtypes(include=['category','object']).columns
    for category in categoric_columns:
        df_dummies = pd.get_dummies(dataframe[category], prefix=f'DUMMY_{category}', drop_first=True).astype(int)
        dataframe = pd.concat([dataframe, df_dummies], axis=1)
        dataframe.drop(category, axis=1, inplace=True)
    return dataframe

In [26]:
df = create_all_dummies(df)

In [27]:
correlation_matrix = df.corr()
correlation_matrix

,EDAD,MOD_RAZONA_CUANTITAT_PUNT,MOD_RAZONA_CUANTITATIVO_PNAL,MOD_RAZONA_CUANTITATIVO_PNBC,MOD_LECTURA_CRITICA_PUNT,MOD_LECTURA_CRITICA_PNAL,MOD_LECTURA_CRITICA_PNBC,MOD_COMPETEN_CIUDADA_PUNT,MOD_COMPETEN_CIUDADA_PNAL,MOD_COMPETEN_CIUDADA_PNBC,MOD_INGLES_PUNT,MOD_INGLES_PNAL,MOD_INGLES_PNBC,MOD_COMUNI_ESCRITA_PUNT,MOD_COMUNI_ESCRITA_DESEM,MOD_COMUNI_ESCRITA_PNAL,MOD_COMUNI_ESCRITA_PNBC,PUNT_GLOBAL,PERCENTIL_GLOBAL,PERCENTIL_NBC,MCPIO_VIVE_ESTUDIA,DUMMY_ESTU_GENERO_M,DUMMY_ESTU_AREARESIDE_Cabecera Municipal,DUMMY_ESTU_ESTADOCIVIL_Separado y/o Viudo,DUMMY_ESTU_ESTADOCIVIL_Soltero,DUMMY_ESTU_ESTADOCIVIL_Unión libre,DUMMY_ESTU_TITULOOBTENIDOBACHILLER_Bachiller pedagógico o normalista,DUMMY_ESTU_TITULOOBTENIDOBACHILLER_Bachiller técnico,DUMMY_ESTU_PAGOMATRICULABECA_Si,DUMMY_ESTU_PAGOMATRICULACREDITO_Si,DUMMY_ESTU_PAGOMATRICULAPADRES_Si,DUMMY_ESTU_PAGOMATRICULAPROPIO_Si,DUMMY_ESTU_COMOCAPACITOEXAMENSB11_Repasó por cuenta propia,DUMMY_ESTU_COMOCAPACITOEXAMENSB11_Tomó un curso de preparación,DUMMY_FAMI_TIENEINTERNET_Si,DUMMY_FAMI_TIENESERVICIOTV_Si,DUMMY_FAMI_TIENECOMPUTADOR_Si,DUMMY_FAMI_TIENELAVADORA_Si,DUMMY_FAMI_TIENEHORNOMICROOGAS_Si,DUMMY_FAMI_TIENEAUTOMOVIL_Si,DUMMY_FAMI_TIENEMOTOCICLETA_Si,DUMMY_FAMI_TIENECONSOLAVIDEOJUEGOS_Si,DUMMY_ESTU_NIVEL_PRGM_ACADEMICO_TÉCNICO PROFESIONAL,DUMMY_ESTU_METODO_PRGM_DISTANCIA VITUAL,DUMMY_ESTU_METODO_PRGM_PRESENCIAL,DUMMY_ESTU_MCPIO_PRESENTACION_BOGOTÁ D.C.,DUMMY_ESTU_MCPIO_PRESENTACION_NILO,DUMMY_ESTU_DEPTO_PRESENTACION_BOGOTÁ,DUMMY_ESTU_DEPTO_PRESENTACION_CUNDINAMARCA,DUMMY_GROUP_ESTU_DEPTO_RESIDE_Grupo 2,DUMMY_GROUP_ESTU_DEPTO_RESIDE_Grupo 3,DUMMY_GROUP_ESTU_DEPTO_RESIDE_Grupo 4,DUMMY_GROUP_ESTU_MCPIO_RESIDE_Grupo 2,DUMMY_GROUP_ESTU_MCPIO_RESIDE_Grupo 3,DUMMY_GROUP_ESTU_MCPIO_RESIDE_Grupo 4,DUMMY_GROUP_ESTU_TIPODOCUMENTOSB11_Grupo 2,DUMMY_GROUP_ESTU_TIPODOCUMENTOSB11_Grupo 3,DUMMY_GROUP_ESTU_TIPODOCUMENTOSB11_Grupo 4,DUMMY_GROUP_ESTU_SEMESTRECURSA_Grupo 2,DUMMY_GROUP_ESTU_SEMESTRECURSA_Grupo 3,DUMMY_GROUP_ESTU_SEMESTRECURSA_Grupo 4,DUMMY_GROUP_FAMI_EDUCACIONPADRE_Grupo 2,DUMMY_GROUP_FAMI_EDUCACIONPADRE_Grupo 3,DUMMY_GROUP_FAMI_EDUCACIONPADRE_Grupo 4,DUMMY_GROUP_FAMI_EDUCACIONMADRE_Grupo 2,DUMMY_GROUP_FAMI_EDUCACIONMADRE_Grupo 3,DUMMY_GROUP_FAMI_EDUCACIONMADRE_Grupo 4,DUMMY_GROUP_FAMI_TRABAJOLABORPADRE_Grupo 2,DUMMY_GROUP_FAMI_TRABAJOLABORPADRE_Grupo 3,DUMMY_GROUP_FAMI_TRABAJOLABORPADRE_Grupo 4,DUMMY_GROUP_FAMI_TRABAJOLABORMADRE_Grupo 2,DUMMY_GROUP_FAMI_TRABAJOLABORMADRE_Grupo 3,DUMMY_GROUP_FAMI_TRABAJOLABORMADRE_Grupo 4,DUMMY_GROUP_FAMI_ESTRATOVIVIENDA_Grupo 2,DUMMY_GROUP_FAMI_ESTRATOVIVIENDA_Grupo 3,DUMMY_GROUP_FAMI_ESTRATOVIVIENDA_Grupo 4,DUMMY_GROUP_FAMI_CUANTOSCOMPARTEBAÑO_Grupo 2,DUMMY_GROUP_FAMI_CUANTOSCOMPARTEBAÑO_Grupo 3,DUMMY_GROUP_FAMI_CUANTOSCOMPARTEBAÑO_Grupo 4,DUMMY_GROUP_ESTU_VALORMATRICULAUNIVERSIDAD_Grupo 2,DUMMY_GROUP_ESTU_VALORMATRICULAUNIVERSIDAD_Grupo 3,DUMMY_GROUP_ESTU_VALORMATRICULAUNIVERSIDAD_Grupo 4,DUMMY_GROUP_ESTU_HORASSEMANATRABAJA_Grupo 2,DUMMY_GROUP_ESTU_HORASSEMANATRABAJA_Grupo 3,DUMMY_GROUP_ESTU_HORASSEMANATRABAJA_Grupo 4,DUMMY_GROUP_INST_NOMBRE_INSTITUCION_Grupo 2,DUMMY_GROUP_INST_NOMBRE_INSTITUCION_Grupo 3,DUMMY_GROUP_INST_NOMBRE_INSTITUCION_Grupo 4,DUMMY_GROUP_ESTU_PRGM_ACADEMICO_Grupo 2,DUMMY_GROUP_ESTU_PRGM_ACADEMICO_Grupo 3,DUMMY_GROUP_ESTU_PRGM_ACADEMICO_Grupo 4,DUMMY_GROUP_GRUPOREFERENCIA_Grupo 2,DUMMY_GROUP_GRUPOREFERENCIA_Grupo 3,DUMMY_GROUP_GRUPOREFERENCIA_Grupo 4,DUMMY_GROUP_ESTU_PRGM_MUNICIPIO_Grupo 2,DUMMY_GROUP_ESTU_PRGM_MUNICIPIO_Grupo 3,DUMMY_GROUP_ESTU_PRGM_MUNICIPIO_Grupo 4,DUMMY_GROUP_ESTU_PRGM_DEPARTAMENTO_Grupo 2,DUMMY_GROUP_ESTU_PRGM_DEPARTAMENTO_Grupo 3,DUMMY_GROUP_ESTU_PRGM_DEPARTAMENTO_Grupo 4,DUMMY_GROUP_ESTU_NUCLEO_PREGRADO_Grupo 2,DUMMY_GROUP_ESTU_NUCLEO_PREGRADO_Grupo 3,DUMMY_GROUP_ESTU_NUCLEO_PREGRADO_Grupo 4,DUMMY_GROUP_ESTU_INST_MUNICIPIO_Grupo 2,DUMMY_GROUP_ESTU_INST_MUNICIPIO_Grupo 3,DUMMY_GROUP_ESTU_INST_MUNICIPIO_Grupo 4,DUMMY_GROUP_ESTU_INST_DEPARTAMENTO_Grupo 2,DUMMY_GROUP_ESTU_INST_DEPARTAMENTO_Grupo 3,DUMMY_GROUP_ESTU_I

In [28]:
positive_correlation = correlation_matrix[correlation_matrix['MOD_INGLES_PUNT'] > 0.4].index.tolist()
positive_correlation.remove('MOD_INGLES_PUNT')
negative_correlation = correlation_matrix[correlation_matrix['MOD_INGLES_PUNT'] < -0.4].index.tolist()

In [29]:
positive_correlation

['MOD_LECTURA_CRITICA_PUNT',
 'MOD_LECTURA_CRITICA_PNAL',
 'MOD_LECTURA_CRITICA_PNBC',
 'MOD_COMPETEN_CIUDADA_PUNT',
 'MOD_COMPETEN_CIUDADA_PNAL',
 'MOD_COMPETEN_CIUDADA_PNBC',
 'MOD_INGLES_PNAL',
 'MOD_INGLES_PNBC',
 'PUNT_GLOBAL',
 'PERCENTIL_GLOBAL',
 'PERCENTIL_NBC']

In [30]:
negative_correlation

[]

In [31]:
def append_data(lists, data):
    for item in data:
        lists.append(item)
    return lists
def delete_data(lists, data):
    for item in data:
        lists.remove(item)
    return lists

In [32]:
impact_to_ingles = []
impact_to_ingles = append_data(impact_to_ingles, positive_correlation)
impact_to_ingles = append_data(impact_to_ingles, negative_correlation)
impact_to_ingles.remove('MOD_LECTURA_CRITICA_PNAL')
impact_to_ingles.remove('MOD_LECTURA_CRITICA_PNBC')
impact_to_ingles.remove('MOD_COMPETEN_CIUDADA_PNBC')
impact_to_ingles.remove('MOD_COMPETEN_CIUDADA_PNAL')
impact_to_ingles.remove('MOD_INGLES_PNAL')
impact_to_ingles.remove('MOD_INGLES_PNBC')
impact_to_ingles.remove('PERCENTIL_GLOBAL')
impact_to_ingles.remove('PERCENTIL_NBC')
impact_to_ingles.remove('MOD_LECTURA_CRITICA_PUNT')
impact_to_ingles.remove('MOD_COMPETEN_CIUDADA_PUNT')

In [33]:
y = df['MOD_INGLES_PUNT']
x = df[impact_to_ingles]
x = sm.add_constant(x)
model = sm.OLS(y,x).fit()

In [34]:
model.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:        MOD_INGLES_PUNT   R-squared:                       0.473
Model:                            OLS   Adj. R-squared:                  0.473
Method:                 Least Squares   F-statistic:                 6.180e+04
Date:                Mon, 25 Sep 2023   Prob (F-statistic):               0.00
Time:                        23:14:57   Log-Likelihood:            -3.0440e+05
No. Observations:               68725   AIC:                         6.088e+05
Df Residuals:                   68723   BIC:                         6.088e+05
Df Model:                           1                                         
Covariance Type:            nonrobust                                         
===============================================================================
                  coef    std err          t      P>|t|      [0.025      0.975]
-------------------------------------------------------------------------------
const           9.6459      0.367     26.266      0.000       8.926      10.366
PUNT_GLOBAL     0.9576      0.004    248.591      0.000       0.950       0.965
==============================================================================
Omnibus:                    11432.927   Durbin-Watson:                   1.978
Prob(Omnibus):                  0.000   Jarque-Bera (JB):            37463.724
Skew:                          -0.847   Prob(JB):                         0.00
Kurtosis:                       6.196   Cond. No.                         452.
==============================================================================

Notes:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
"""

In [35]:
x_train, x_test, y_train, y_test = train_test_split(x,y, test_size=0.2)

In [36]:
x_train = sm.add_constant(x_train)
model = sm.OLS(y_train, x_train).fit()
print(model.summary())

                            OLS Regression Results                            
Dep. Variable:        MOD_INGLES_PUNT   R-squared:                       0.473
Model:                            OLS   Adj. R-squared:                  0.473
Method:                 Least Squares   F-statistic:                 4.928e+04
Date:                Mon, 25 Sep 2023   Prob (F-statistic):               0.00
Time:                        23:14:57   Log-Likelihood:            -2.4351e+05
No. Observations:               54980   AIC:                         4.870e+05
Df Residuals:                   54978   BIC:                         4.870e+05
Df Model:                           1                                         
Covariance Type:            nonrobust                                         
                  coef    std err          t      P>|t|      [0.025      0.975]
-------------------------------------------------------------------------------
const           9.7015      0.411     23.620      

In [37]:
y_hat_train = model.predict(x_train)
y_hat = model.predict(x_test)
mse_train = mean_squared_error(y_train, y_hat_train)
mse_test = mean_squared_error(y_test, y_hat)
print(f'Error calculado en train, test: {mse_train}, {mse_test}')

Error calculado en train, test: 411.7233019877419, 412.5227472049505


In [38]:
x = df
x = sm.add_constant(x)
model = sm.OLS(y,x).fit()

In [39]:
print(model.summary())

                            OLS Regression Results                            
Dep. Variable:        MOD_INGLES_PUNT   R-squared:                       1.000
Model:                            OLS   Adj. R-squared:                  1.000
Method:                 Least Squares   F-statistic:                 1.241e+31
Date:                Mon, 25 Sep 2023   Prob (F-statistic):               0.00
Time:                        23:14:59   Log-Likelihood:             1.9136e+06
No. Observations:               68725   AIC:                        -3.827e+06
Df Residuals:                   68611   BIC:                        -3.826e+06
Df Model:                         113                                         
Covariance Type:            nonrobust                                         
                                                                           coef    std err          t      P>|t|      [0.025      0.975]
-------------------------------------------------------------------------

In [40]:
x_train, x_test, y_train, y_test = train_test_split(x,y, test_size=0.2)
x_train = sm.add_constant(x_train)
model = sm.OLS(y_train, x_train).fit()
print(model.summary())

                            OLS Regression Results                            
Dep. Variable:        MOD_INGLES_PUNT   R-squared:                       1.000
Model:                            OLS   Adj. R-squared:                  1.000
Method:                 Least Squares   F-statistic:                 1.527e+31
Date:                Mon, 25 Sep 2023   Prob (F-statistic):               0.00
Time:                        23:15:02   Log-Likelihood:             1.5426e+06
No. Observations:               54980   AIC:                        -3.085e+06
Df Residuals:                   54866   BIC:                        -3.084e+06
Df Model:                         113                                         
Covariance Type:            nonrobust                                         
                                                                           coef    std err          t      P>|t|      [0.025      0.975]
-------------------------------------------------------------------------

In [41]:
y_hat_train = model.predict(x_train)
y_hat = model.predict(x_test)
mse_train = mean_squared_error(y_train, y_hat_train)
mse_test = mean_squared_error(y_test, y_hat)
print(f'Error calculado en train, test: {mse_train}, {mse_test}')

Error calculado en train, test: 2.4912958209284645e-26, 2.5048806488527753e-26


In [42]:
new_values = ['EDAD','MCPIO_VIVE_ESTUDIA', 'DUMMY_ESTU_AREARESIDE_Cabecera Municipal', 'DUMMY_ESTU_ESTADOCIVIL_Soltero', 'DUMMY_ESTU_ESTADOCIVIL_Unión libre', 'DUMMY_ESTU_TITULOOBTENIDOBACHILLER_Bachiller técnico', 'DUMMY_ESTU_PAGOMATRICULAPADRES_Si', 'DUMMY_ESTU_COMOCAPACITOEXAMENSB11_Repasó por cuenta propia', 'DUMMY_ESTU_COMOCAPACITOEXAMENSB11_Tomó un curso de preparación', 'DUMMY_FAMI_TIENEINTERNET_Si', 'DUMMY_FAMI_TIENESERVICIOTV_Si', 'DUMMY_FAMI_TIENECOMPUTADOR_Si', 'DUMMY_FAMI_TIENELAVADORA_Si', 'DUMMY_FAMI_TIENEHORNOMICROOGAS_Si', 'DUMMY_FAMI_TIENEAUTOMOVIL_Si', 'DUMMY_FAMI_TIENEMOTOCICLETA_Si', 'DUMMY_ESTU_METODO_PRGM_PRESENCIAL', 'DUMMY_ESTU_MCPIO_PRESENTACION_BOGOTÁ D.C.', 'DUMMY_ESTU_DEPTO_PRESENTACION_BOGOTÁ', 'DUMMY_GROUP_ESTU_DEPTO_RESIDE_Grupo 2', 'DUMMY_GROUP_ESTU_MCPIO_RESIDE_Grupo 4', 'DUMMY_GROUP_ESTU_SEMESTRECURSA_Grupo 3', 'DUMMY_GROUP_ESTU_SEMESTRECURSA_Grupo 4', 'DUMMY_GROUP_FAMI_EDUCACIONPADRE_Grupo 4', 'DUMMY_GROUP_FAMI_EDUCACIONMADRE_Grupo 4', 'DUMMY_GROUP_FAMI_TRABAJOLABORPADRE_Grupo 3', 'DUMMY_GROUP_FAMI_TRABAJOLABORPADRE_Grupo 4','DUMMY_GROUP_FAMI_TRABAJOLABORMADRE_Grupo 4', 'DUMMY_GROUP_FAMI_ESTRATOVIVIENDA_Grupo 4', 'DUMMY_GROUP_FAMI_CUANTOSCOMPARTEBAÑO_Grupo 4', 'DUMMY_GROUP_ESTU_VALORMATRICULAUNIVERSIDAD_Grupo 4', 'DUMMY_GROUP_ESTU_HORASSEMANATRABAJA_Grupo 4', 'DUMMY_GROUP_INST_NOMBRE_INSTITUCION_Grupo 2', 'DUMMY_GROUP_INST_NOMBRE_INSTITUCION_Grupo 4', 'DUMMY_GROUP_ESTU_PRGM_ACADEMICO_Grupo 4', 'DUMMY_GROUP_GRUPOREFERENCIA_Grupo 3', 'DUMMY_GROUP_GRUPOREFERENCIA_Grupo 4', 'DUMMY_GROUP_ESTU_PRGM_MUNICIPIO_Grupo 2', 'DUMMY_GROUP_ESTU_PRGM_MUNICIPIO_Grupo 3', 'DUMMY_GROUP_ESTU_PRGM_MUNICIPIO_Grupo 4', 'DUMMY_GROUP_ESTU_PRGM_DEPARTAMENTO_Grupo 2', 'DUMMY_GROUP_ESTU_PRGM_DEPARTAMENTO_Grupo 3', 'DUMMY_GROUP_ESTU_PRGM_DEPARTAMENTO_Grupo 4', 'DUMMY_GROUP_ESTU_NUCLEO_PREGRADO_Grupo 2', 'DUMMY_GROUP_ESTU_NUCLEO_PREGRADO_Grupo 3', 'DUMMY_GROUP_ESTU_NUCLEO_PREGRADO_Grupo 4', 'DUMMY_GROUP_ESTU_INST_MUNICIPIO_Grupo 2', 'DUMMY_GROUP_ESTU_INST_MUNICIPIO_Grupo 3', 'DUMMY_GROUP_ESTU_INST_DEPARTAMENTO_Grupo 2', 'DUMMY_GROUP_ESTU_INST_DEPARTAMENTO_Grupo 4', 'DUMMY_GROUP_INST_CARACTER_ACADEMICO_Grupo 2', 'DUMMY_GROUP_INST_CARACTER_ACADEMICO_Grupo 4', 'DUMMY_GROUP_INST_ORIGEN_Grupo 2', 'DUMMY_GROUP_INST_ORIGEN_Grupo 3', 'DUMMY_GROUP_INST_ORIGEN_Grupo 4']

In [43]:
impact_to_ingles = append_data(impact_to_ingles, new_values)

In [44]:
x = df[impact_to_ingles]
x = sm.add_constant(x)
model = sm.OLS(y,x).fit()

In [45]:
print(model.summary())

                            OLS Regression Results                            
Dep. Variable:        MOD_INGLES_PUNT   R-squared:                       0.533
Model:                            OLS   Adj. R-squared:                  0.533
Method:                 Least Squares   F-statistic:                     1427.
Date:                Mon, 25 Sep 2023   Prob (F-statistic):               0.00
Time:                        23:15:03   Log-Likelihood:            -3.0026e+05
No. Observations:               68725   AIC:                         6.006e+05
Df Residuals:                   68669   BIC:                         6.011e+05
Df Model:                          55                                         
Covariance Type:            nonrobust                                         
                                                                     coef    std err          t      P>|t|      [0.025      0.975]
-------------------------------------------------------------------------------

In [46]:
x_train, x_test, y_train, y_test = train_test_split(x,y, test_size=0.2)
x_train = sm.add_constant(x_train)
model = sm.OLS(y_train, x_train).fit()
print(model.summary())

                            OLS Regression Results                            
Dep. Variable:        MOD_INGLES_PUNT   R-squared:                       0.535
Model:                            OLS   Adj. R-squared:                  0.534
Method:                 Least Squares   F-statistic:                     1148.
Date:                Mon, 25 Sep 2023   Prob (F-statistic):               0.00
Time:                        23:15:04   Log-Likelihood:            -2.4014e+05
No. Observations:               54980   AIC:                         4.804e+05
Df Residuals:                   54924   BIC:                         4.809e+05
Df Model:                          55                                         
Covariance Type:            nonrobust                                         
                                                                     coef    std err          t      P>|t|      [0.025      0.975]
-------------------------------------------------------------------------------

In [47]:
y_hat_train = model.predict(x_train)
y_hat = model.predict(x_test)
mse_train = mean_squared_error(y_train, y_hat_train)
mse_test = mean_squared_error(y_test, y_hat)
print(f'Error calculado en train, test: {mse_train}, {mse_test}')

Error calculado en train, test: 364.22926763921305, 368.7153893244717


In [48]:
values_remove = ['DUMMY_ESTU_AREARESIDE_Cabecera Municipal', 'DUMMY_ESTU_ESTADOCIVIL_Unión libre', 'DUMMY_ESTU_TITULOOBTENIDOBACHILLER_Bachiller técnico', 'DUMMY_ESTU_COMOCAPACITOEXAMENSB11_Tomó un curso de preparación', 'DUMMY_FAMI_TIENEINTERNET_Si', 'DUMMY_ESTU_MCPIO_PRESENTACION_BOGOTÁ D.C.', 'DUMMY_ESTU_DEPTO_PRESENTACION_BOGOTÁ', 'DUMMY_GROUP_ESTU_SEMESTRECURSA_Grupo 3', 'DUMMY_GROUP_ESTU_SEMESTRECURSA_Grupo 4', 'DUMMY_GROUP_INST_NOMBRE_INSTITUCION_Grupo 2', 'DUMMY_GROUP_INST_NOMBRE_INSTITUCION_Grupo 4', 'DUMMY_GROUP_GRUPOREFERENCIA_Grupo 3', 'DUMMY_GROUP_GRUPOREFERENCIA_Grupo 4', 'DUMMY_GROUP_ESTU_PRGM_MUNICIPIO_Grupo 2', 'DUMMY_GROUP_ESTU_PRGM_MUNICIPIO_Grupo 3', 'DUMMY_GROUP_ESTU_PRGM_MUNICIPIO_Grupo 4', 'DUMMY_GROUP_ESTU_PRGM_DEPARTAMENTO_Grupo 2', 'DUMMY_GROUP_ESTU_PRGM_DEPARTAMENTO_Grupo 3', 'DUMMY_GROUP_ESTU_PRGM_DEPARTAMENTO_Grupo 4', 'DUMMY_GROUP_ESTU_NUCLEO_PREGRADO_Grupo 2', 'DUMMY_GROUP_ESTU_NUCLEO_PREGRADO_Grupo 3', 'DUMMY_GROUP_ESTU_NUCLEO_PREGRADO_Grupo 4', 'DUMMY_GROUP_ESTU_INST_MUNICIPIO_Grupo 2', 'DUMMY_GROUP_ESTU_INST_MUNICIPIO_Grupo 3', 'DUMMY_GROUP_ESTU_INST_DEPARTAMENTO_Grupo 2', 'DUMMY_GROUP_ESTU_INST_DEPARTAMENTO_Grupo 4', 'DUMMY_GROUP_INST_CARACTER_ACADEMICO_Grupo 2', 'DUMMY_GROUP_INST_ORIGEN_Grupo 3']

In [49]:
impact_to_ingles = delete_data(impact_to_ingles,values_remove)

In [50]:
x = df[impact_to_ingles]
x = sm.add_constant(x)
model = sm.OLS(y,x).fit()

In [51]:
print(model.summary())

                            OLS Regression Results                            
Dep. Variable:        MOD_INGLES_PUNT   R-squared:                       0.532
Model:                            OLS   Adj. R-squared:                  0.532
Method:                 Least Squares   F-statistic:                     2790.
Date:                Mon, 25 Sep 2023   Prob (F-statistic):               0.00
Time:                        23:15:04   Log-Likelihood:            -3.0035e+05
No. Observations:               68725   AIC:                         6.008e+05
Df Residuals:                   68696   BIC:                         6.010e+05
Df Model:                          28                                         
Covariance Type:            nonrobust                                         
                                                                 coef    std err          t      P>|t|      [0.025      0.975]
-----------------------------------------------------------------------------------

In [52]:
x_train, x_test, y_train, y_test = train_test_split(x,y, test_size=0.2)
x_train = sm.add_constant(x_train)
model = sm.OLS(y_train, x_train).fit()
print(model.summary())

                            OLS Regression Results                            
Dep. Variable:        MOD_INGLES_PUNT   R-squared:                       0.531
Model:                            OLS   Adj. R-squared:                  0.530
Method:                 Least Squares   F-statistic:                     2218.
Date:                Mon, 25 Sep 2023   Prob (F-statistic):               0.00
Time:                        23:15:05   Log-Likelihood:            -2.4023e+05
No. Observations:               54980   AIC:                         4.805e+05
Df Residuals:                   54951   BIC:                         4.808e+05
Df Model:                          28                                         
Covariance Type:            nonrobust                                         
                                                                 coef    std err          t      P>|t|      [0.025      0.975]
-----------------------------------------------------------------------------------

In [53]:
y_hat_train = model.predict(x_train)
y_hat = model.predict(x_test)
mse_train = mean_squared_error(y_train, y_hat_train)
mse_test = mean_squared_error(y_test, y_hat)
print(f'Error calculado en train, test: {mse_train}, {mse_test}')

Error calculado en train, test: 365.38206101824244, 368.8696995837154


In [54]:
print('Las variables mas importantes para predecir el puntaje fueron: ', impact_to_ingles)

Las variables mas importantes para predecir el puntaje fueron:  ['PUNT_GLOBAL', 'EDAD', 'MCPIO_VIVE_ESTUDIA', 'DUMMY_ESTU_ESTADOCIVIL_Soltero', 'DUMMY_ESTU_PAGOMATRICULAPADRES_Si', 'DUMMY_ESTU_COMOCAPACITOEXAMENSB11_Repasó por cuenta propia', 'DUMMY_FAMI_TIENESERVICIOTV_Si', 'DUMMY_FAMI_TIENECOMPUTADOR_Si', 'DUMMY_FAMI_TIENELAVADORA_Si', 'DUMMY_FAMI_TIENEHORNOMICROOGAS_Si', 'DUMMY_FAMI_TIENEAUTOMOVIL_Si', 'DUMMY_FAMI_TIENEMOTOCICLETA_Si', 'DUMMY_ESTU_METODO_PRGM_PRESENCIAL', 'DUMMY_GROUP_ESTU_DEPTO_RESIDE_Grupo 2', 'DUMMY_GROUP_ESTU_MCPIO_RESIDE_Grupo 4', 'DUMMY_GROUP_FAMI_EDUCACIONPADRE_Grupo 4', 'DUMMY_GROUP_FAMI_EDUCACIONMADRE_Grupo 4', 'DUMMY_GROUP_FAMI_TRABAJOLABORPADRE_Grupo 3', 'DUMMY_GROUP_FAMI_TRABAJOLABORPADRE_Grupo 4', 'DUMMY_GROUP_FAMI_TRABAJOLABORMADRE_Grupo 4', 'DUMMY_GROUP_FAMI_ESTRATOVIVIENDA_Grupo 4', 'DUMMY_GROUP_FAMI_CUANTOSCOMPARTEBAÑO_Grupo 4', 'DUMMY_GROUP_ESTU_VALORMATRICULAUNIVERSIDAD_Grupo 4', 'DUMMY_GROUP_ESTU_HORASSEMANATRABAJA_Grupo 4', 'DUMMY_GROUP_ESTU_PRG